<a href="https://colab.research.google.com/github/TarasMuzychuk/Practise_Task-3/blob/main/Task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=efd5bdda5f883d645ec912bf124ec4c37d3b540b07c9149840fbd356ebd411f5
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


# Робота з Spark SQL
## Складна аналітика з Spark SQL

> Можна виконати або за допомогою чистого `DataFrame` API, `Pandas on Spark` API, або за допомогою чистих `SQL` запитів. Вибір за вами.

Ви розробник у компанії **BikeServe**, яка займається орендою велосипедів/скутерів. У вас є певні місця ("станції"), де зберігаються ваші велосипеди. Якщо на станції немає вільних місць для велосипедів (хтось уже зарезервував або взяв велосипед) протягом певного періоду часу (`timeslot`), це означає, що бізнес йде чудово. Однак вам потрібно покращити обслуговування клієнтів, пропонуючи користувачам велосипеди, коли та в тому місці, де це для них найважливіше.

Ваше завдання — знайти найбільш важливі ("критичні") пари станції та періоду часу `(stationId, timeslot)`, щоб ваш бізнес знав, куди і коли доставити більше велосипедів.

Ваш результат має бути відсортований за цією *критичністю* у порядку спадання.

Набір даних містить:
* `register.csv` містить інформацію з вашої IoT системи моніторингу про кількість використаних і вільних слотів на ваших станціях оренди велосипедів. Кожен рядок відповідає одному запису про ситуацію на одній станції в певний момент часу.

    Кожен рядок має такий формат:

    ```bash
    stationId\ttimestamp\tusedslots\tfreeslots
    ```
    
    де `timestamp` має формат `datetime`.

    > Перший рядок файлу містить заголовок.
    > Деякі дані в наборі даних пошкоджено через тимчасові збої мережі та/або вашої системи моніторингу. Це означає, що деякі рядки характеризуються "використаними слотами (used slots) = 0" і "вільними слотами (free slots) = 0". **Ці рядки необхідно відфільтрувати** перед виконанням будь-яких операцій.

* `input/stations.csv` містить опис станцій.

    Кожен рядок має такий формат:

    ```bash
    stationId\tlongitude\ttitude\tname
    ```
    > Перший рядок файлу містить заголовок.

### Опис завдання

Кожна пара "день тижня – година" є "часовим інтервалом" (`timeslot`) і пов’язана з усіма показаннями моніторингу, пов’язаними з цією парою, незалежно від дати. Наприклад, часовий інтервал `Wednesday - 17` відповідає всім показанням, зробленим у середу з `17:00:00` до `17:59:59`.

Станція $S_i$ знаходиться в критичному стані, якщо кількість вільних слотів дорівнює `0` (всі велосипеди на станції заброньовані).

*Критичність* станції $S_i$ у часовому інтервалі $T_j$ визначається як:

$$
\frac{\text{кількість записів із числом вільних слотів, яке дорівнює нулю, для пари}_{\left(S_i,T_j\right)}}{\text{загальна кількість записів для пари}_{\left(S_i,T_j\right)}}
$$

необхідно:
* Обчислити значення *критичності* для кожної пари $(S_i, T_j)$.
* Вибирати лише пари, у яких значення *критичності* перевищує "мінімальний поріг критичності".
    * `Мінімальний поріг критичності` має бути параметром конфігурації програми.
* Зберегти у вихідній папці вибрані записи, використовуючи файли `csv` (із заголовком). Зберегти лише такі атрибути:
    * ідентифікатор станції
    * день тижня
    * година
    * критичність
    * довгота станції
    * широта станції
* Зберегти результати за зменшення критичності. Якщо є два або більше записів, що характеризуються однаковим значенням критичності, то додатково відсортувати по ідентифікатору станції (у порядку зростання). Якщо і станція та сама, то сортувати за днем тижня (за зростанням) і, нарешті, за годиною (за зростанням).

### Поради та підказки

Мова SQL, доступна в Spark SQL, має низку попередньо визначених функцій, одна з яких, `hour(timestamp)`, може використовуватися в запитах SQL або в перетворенні `selectExpr`, щоб вибрати `hour` з заданого позначка часу. Ще одна цікава функція, `date_format(timestamp,format)`, може бути використана для отримання іншої корисної інформації зі стовпця timestamp. Наприклад, у форматі `EE` можна отримати день тижня.

```python
new_df= df.selectExpr("date_format(timestamp,'EE') as weekday hour(timestamp) as hour")
```

Щоб вказати, що роздільником вхідних файлів CSV є спеціальний символ `tab`, установіть параметр роздільника на `\\t`, викликавши `.option("delimiter", "\\t")` під час читання вхідних даних.

## Конфігурація

- `number_cores`: Кількість ядер, виділених під Spark
- `memory_gb`: Обʼєм оперативної памʼяті, виділеної під Spark (в Гб)

In [2]:
number_cores = 2
memory_gb = 4

In [3]:
from pyspark.sql import SparkSession
import warnings

warnings.filterwarnings('ignore')

spark = (SparkSession
    .builder
    .appName('Spark Bikes')
    .master(f"local[{number_cores}]")
    .config("spark.driver.memory", f"{memory_gb}g")
    .getOrCreate())

## Рішення
Прочитайте вміст вхідного файлу `register.csv` і збережіть його у DataFrame.

Вхідний файл має заголовок.

Схема даних:
* station: integer (nullable = true)
* timestamp: timestamp (nullable = true)
* used_slots: integer (nullable = true)
* free_slots: integer (nullable = true)

In [5]:
# ваш код
from pyspark.sql import SparkSession

# Ініціалізуємо SparkSession
spark = SparkSession.builder \
    .appName("Read Register CSV") \
    .getOrCreate()

# Створюємо тимчасовий вид таблиці з файлу CSV
spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", "\t") \
    .load("register.csv") \
    .createOrReplaceTempView("register")

# Виконуємо SQL-запит для вибору всіх даних з тимчасової таблиці
register_df = spark.sql("SELECT * FROM register")

# Виводимо результат
register_df.show()



+-------+-------------------+----------+----------+
|station|          timestamp|used_slots|free_slots|
+-------+-------------------+----------+----------+
|      1|2023-05-15 12:01:00|         0|        18|
|      1|2023-05-15 12:02:00|         0|        18|
|      1|2023-05-15 12:04:00|         0|        18|
|      1|2023-05-15 12:06:00|         0|        18|
|      1|2023-05-15 12:08:00|         0|        18|
|      1|2023-05-15 12:10:00|         0|        18|
|      1|2023-05-15 12:12:00|         0|        18|
|      1|2023-05-15 12:14:00|         0|        18|
|      1|2023-05-15 12:16:00|         0|        18|
|      1|2023-05-15 12:18:00|         0|        18|
|      1|2023-05-15 12:20:00|         2|        16|
|      1|2023-05-15 12:22:00|         3|        15|
|      1|2023-05-15 12:24:00|         3|        15|
|      1|2023-05-15 12:26:00|         3|        15|
|      1|2023-05-15 12:28:00|         4|        14|
|      1|2023-05-15 12:30:00|         0|        12|
|      1|202

Видаліть рядки де одночасно `free_slots = 0` та `used_slots = 0`

In [6]:
# ваш код
# Фільтруємо рядки, де free_slots не дорівнює 0 та used_slots не дорівнює 0
register_df_is_filtered = spark.sql("""
    SELECT *
    FROM register
    WHERE free_slots != 0 AND used_slots != 0
""")

# Виводимо для перевірки
register_df_is_filtered.show()


+-------+-------------------+----------+----------+
|station|          timestamp|used_slots|free_slots|
+-------+-------------------+----------+----------+
|      1|2023-05-15 12:20:00|         2|        16|
|      1|2023-05-15 12:22:00|         3|        15|
|      1|2023-05-15 12:24:00|         3|        15|
|      1|2023-05-15 12:26:00|         3|        15|
|      1|2023-05-15 12:28:00|         4|        14|
|      1|2023-05-15 12:32:00|         4|        14|
|      1|2023-05-15 12:34:00|         4|        14|
|      1|2023-05-15 12:36:00|         4|        14|
|      1|2023-05-15 12:38:00|         4|        14|
|      1|2023-05-15 12:40:00|         5|        13|
|      1|2023-05-15 12:42:00|         6|        12|
|      1|2023-05-15 12:46:00|         6|        12|
|      1|2023-05-15 12:48:00|         7|        11|
|      1|2023-05-15 12:50:00|         7|        11|
|      1|2023-05-15 12:52:00|         7|        11|
|      1|2023-05-15 12:54:00|         8|        10|
|      1|202

Нам потрібен логічний маркер, щоб побачити, заповнена станція чи ні. Це можна зробити за допомогою UDF під назвою `full(free_slots: int)`, яка повертає
* 1, якщо `free_slots` дорівнює 0
* 0, якщо `free_slots` більше 0

> Якщо ви використовуєте Pandas on Spark API, то треба самостійно застосувати цю функцію (або переписати її)

In [8]:
# ваш код
from pyspark.sql.types import IntegerType

# Визначаємо SQL функцію для перевірки стану станції
spark.udf.register("full_function", lambda free_slots: 1 if free_slots == 0 else 0, IntegerType())

# Додаємо новий стовпець, який використовує SQL функцію
register_df_is_filtered.createOrReplaceTempView("register_with_udf")
register_with_is_full = spark.sql("""
    SELECT *, full_function(free_slots) AS full
    FROM register_with_udf
""")

# Виводимо результат
register_with_is_full.show()


+-------+-------------------+----------+----------+----+
|station|          timestamp|used_slots|free_slots|full|
+-------+-------------------+----------+----------+----+
|      1|2023-05-15 12:20:00|         2|        16|   0|
|      1|2023-05-15 12:22:00|         3|        15|   0|
|      1|2023-05-15 12:24:00|         3|        15|   0|
|      1|2023-05-15 12:26:00|         3|        15|   0|
|      1|2023-05-15 12:28:00|         4|        14|   0|
|      1|2023-05-15 12:32:00|         4|        14|   0|
|      1|2023-05-15 12:34:00|         4|        14|   0|
|      1|2023-05-15 12:36:00|         4|        14|   0|
|      1|2023-05-15 12:38:00|         4|        14|   0|
|      1|2023-05-15 12:40:00|         5|        13|   0|
|      1|2023-05-15 12:42:00|         6|        12|   0|
|      1|2023-05-15 12:46:00|         6|        12|   0|
|      1|2023-05-15 12:48:00|         7|        11|   0|
|      1|2023-05-15 12:50:00|         7|        11|   0|
|      1|2023-05-15 12:52:00|  

Створіть DataFrame з такою схемою:
* station: integer (nullable = true)
* dayofweek: string (nullable = true)
* hour: integer (nullable = true)
* fullstatus: integer (nullable = true) - 1 = full, 0 = non-full

In [20]:
# ваш код
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, hour, minute, second, concat_ws
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Ініціалізуємо SparkSession
spark = SparkSession.builder \
    .appName("Create DataFrame") \
    .getOrCreate()

# Описуємо схему для DataFrame
napolnenie = StructType([
    StructField("station", IntegerType(), nullable=True),
    StructField("hour", StringType(), nullable=True)
])

# Створюємо DataFrame з вказаною схемою
register_df = spark.createDataFrame([], napolnenie )

# Додаємо необхідні стовпці до DataFrame
register_df = register_with_is_full.withColumn("dayofweek", to_date(col("timestamp"))) \
                         .withColumn("hour", hour(col("timestamp"))) \
                         .withColumn("minute", minute(col("timestamp"))) \
                         .withColumn("second", second(col("timestamp"))) \
                         .withColumn("hour",
                                     concat_ws(":", col("hour"), col("minute"), col("second"))) \
                         .withColumn("fullstatus", col("full"))

# Вибираємо тільки необхідні колонки: station, dayofweek, hour, fullstatus
selected_df_of_register_df = register_df.select("station", "dayofweek", "hour", "fullstatus")

# Виводимо вміст DataFrame
selected_df_of_register_df.show()



+-------+----------+-------+----------+
|station| dayofweek|   hour|fullstatus|
+-------+----------+-------+----------+
|      1|2023-05-15|12:20:0|         0|
|      1|2023-05-15|12:22:0|         0|
|      1|2023-05-15|12:24:0|         0|
|      1|2023-05-15|12:26:0|         0|
|      1|2023-05-15|12:28:0|         0|
|      1|2023-05-15|12:32:0|         0|
|      1|2023-05-15|12:34:0|         0|
|      1|2023-05-15|12:36:0|         0|
|      1|2023-05-15|12:38:0|         0|
|      1|2023-05-15|12:40:0|         0|
|      1|2023-05-15|12:42:0|         0|
|      1|2023-05-15|12:46:0|         0|
|      1|2023-05-15|12:48:0|         0|
|      1|2023-05-15|12:50:0|         0|
|      1|2023-05-15|12:52:0|         0|
|      1|2023-05-15|12:54:0|         0|
|      1|2023-05-15|12:56:0|         0|
|      1|2023-05-15|12:58:0|         0|
|      1|2023-05-15| 13:0:0|         0|
|      1|2023-05-15| 13:2:0|         0|
+-------+----------+-------+----------+
only showing top 20 rows



Визначте одну групу для кожної комбінації `(station, dayofweek, hour)`

In [21]:
# ваш код
# Виконуємо реєстрацію тимчасової таблиці для подальшого використання SQL
selected_df_of_register_df.createOrReplaceTempView("register_with_count")

# Формуємо SQL-запит для визначення груп для кожної комбінації (station, dayofweek, hour)
query = """
    SELECT station, dayofweek, hour, COUNT(*) AS group_count
    FROM register_with_count
    GROUP BY station, dayofweek, hour
"""

# Виконуємо SQL-запит і отримуємо результат
groupedby_df = spark.sql(query)

# Виводимо результат
groupedby_df.show()




+-------+----------+-------+-----------+
|station| dayofweek|   hour|group_count|
+-------+----------+-------+-----------+
|      1|2023-05-16| 6:56:0|          1|
|      1|2023-05-16|20:54:0|          1|
|      1|2023-05-17|12:40:0|          1|
|      1|2023-05-17|23:34:0|          1|
|      1|2023-05-18|20:34:0|          1|
|      1|2023-05-19|13:44:0|          1|
|      1|2023-05-20|13:56:0|          1|
|      1|2023-05-20|14:34:0|          1|
|      1|2023-05-20|19:14:0|          1|
|      1|2023-05-21|14:44:0|          1|
|      1|2023-05-22|14:46:0|          1|
|      1|2023-05-22|15:30:0|          1|
|      1|2023-05-22|16:54:0|          1|
|      1|2023-05-22|21:22:0|          1|
|      1|2023-05-23|10:58:0|          1|
|      1|2023-05-24|12:20:0|          1|
|      1|2023-05-24|19:38:0|          1|
|      1|2023-05-25|15:54:0|          1|
|      1|2023-05-25| 17:4:0|          1|
|      1|2023-05-25|17:24:0|          1|
+-------+----------+-------+-----------+
only showing top

Обчисліть "критичність" для кожної групи `(station, dayofweek, hour)`, тобто для кожної пари `(station, timeslot)`.

Критичність дорівнює середньому `fullStatus`.

In [38]:
# ваш код
from pyspark.sql import SparkSession

# Ініціалізуємо SparkSession
spark = SparkSession.builder \
    .appName("Calculate Criticality") \
    .getOrCreate()

# Реєструємо тимчасову таблицю для DataFrame
register_df.createOrReplaceTempView("register")

# Виконуємо SQL запит для обчислення критичності за формулою для кожної групи (станція, день тижня, година)
criticality_df = spark.sql("""
    SELECT
        station,
        dayofweek,
        hour,
        SUM(used_slots) / (SUM(used_slots) + SUM(free_slots)) AS criticality
    FROM
        register
    GROUP BY
        station,
        dayofweek,
        hour
""")

# Виводимо результат
criticality_df.show()



+-------+----------+-------+-------------------+
|station| dayofweek|   hour|        criticality|
+-------+----------+-------+-------------------+
|      1|2023-05-16| 6:56:0| 0.6666666666666666|
|      1|2023-05-16|20:54:0|0.21428571428571427|
|      1|2023-05-17|12:40:0|                0.1|
|      1|2023-05-17|23:34:0| 0.3157894736842105|
|      1|2023-05-18|20:34:0|0.26666666666666666|
|      1|2023-05-19|13:44:0|0.47619047619047616|
|      1|2023-05-20|13:56:0| 0.7142857142857143|
|      1|2023-05-20|14:34:0| 0.7619047619047619|
|      1|2023-05-20|19:14:0|0.05263157894736842|
|      1|2023-05-21|14:44:0| 0.3157894736842105|
|      1|2023-05-22|14:46:0| 0.8333333333333334|
|      1|2023-05-22|15:30:0| 0.7222222222222222|
|      1|2023-05-22|16:54:0| 0.4444444444444444|
|      1|2023-05-22|21:22:0| 0.3888888888888889|
|      1|2023-05-23|10:58:0|0.10526315789473684|
|      1|2023-05-24|12:20:0|0.23809523809523808|
|      1|2023-05-24|19:38:0|                0.2|
|      1|2023-05-25|

Виберіть лише рядки з `criticality > threshold`

> `threshold` є деякою бізнес-вимогою, тому візьміть випадкове число від `0.1` до `0.5`, яке вам подобається :)

In [39]:
threshold = 0.25
# ваш код
# Вибираємо лише рядки, де значення критичності перевищує поріг
selected_criticality_df = criticality_df.filter(criticality_df["criticality"] > threshold)

# Виводимо результат
selected_criticality_df.show()


+-------+----------+-------+-------------------+
|station| dayofweek|   hour|        criticality|
+-------+----------+-------+-------------------+
|      1|2023-05-16| 6:56:0| 0.6666666666666666|
|      1|2023-05-17|23:34:0| 0.3157894736842105|
|      1|2023-05-18|20:34:0|0.26666666666666666|
|      1|2023-05-19|13:44:0|0.47619047619047616|
|      1|2023-05-20|13:56:0| 0.7142857142857143|
|      1|2023-05-20|14:34:0| 0.7619047619047619|
|      1|2023-05-21|14:44:0| 0.3157894736842105|
|      1|2023-05-22|14:46:0| 0.8333333333333334|
|      1|2023-05-22|15:30:0| 0.7222222222222222|
|      1|2023-05-22|16:54:0| 0.4444444444444444|
|      1|2023-05-22|21:22:0| 0.3888888888888889|
|      1|2023-05-25|17:24:0| 0.2631578947368421|
|      1|2023-05-25|18:52:0| 0.3333333333333333|
|      1|2023-05-26|13:34:0| 0.2631578947368421|
|      1|2023-05-26|21:10:0|             0.9375|
|      1|2023-05-27|20:56:0| 0.6470588235294118|
|      1|2023-05-28| 17:4:0|                0.4|
|      1|2023-05-28|

Прочитайте вміст вхідного файлу `stations.csv` і збережіть його у DataFrame.

Вхідний файл має заголовок.

Схема даних:
* id: integer (nullable = true)
* longitude: double (nullable = true)
* latitude: double (nullable = true)
* name: string (nullable = true)

In [40]:
# ваш код
from pyspark.sql import SparkSession

# Ініціалізуємо SparkSession
spark = SparkSession.builder \
    .appName("Read Stations CSV") \
    .getOrCreate()

# Читаємо вміст вхідного файлу stations.csv та зберігаємо його у DataFrame
stations_df = spark.read.csv("stations.csv", header=True, sep="\t")

# Виведемо перші 5 рядків для перевірки
stations_df.show()





+---+---------+---------+--------------------+
| id|longitude| latitude|                name|
+---+---------+---------+--------------------+
|  1| 2.180019|41.397978|Gran Via Corts Ca...|
|  2| 2.176414|41.394381|       Plaza TetuÃ¡n|
|  3| 2.181164| 41.39375|             Ali Bei|
|  4|   2.1814|41.393364|               Ribes|
|  5| 2.180214|41.391072|  Pg LluÃ­s Companys|
|  6| 2.180508|41.391272|  Pg LluÃ­s Companys|
|  7| 2.183183|41.388867|  Pg LluÃ­s Companys|
|  8| 2.183453|41.389044|Passeig lluÃ­s co...|
|  9| 2.185294|41.385006|MarquÃ¨s de l\'Ar...|
| 10| 2.185206|41.384875|Avinguda del Marq...|
| 11| 2.183622|41.385394|             ComerÃ§|
| 12| 2.193939|41.381681|            Trelawny|
| 13| 2.195661|41.384522|pg marÃ­tim barce...|
| 14| 2.195603|41.384417|     Passeig Maritim|
| 15| 2.195706|41.386811|       Avda. Litoral|
| 16| 2.195764|41.386869|    Avinguda Litoral|
| 17| 2.170775|41.395161|              Girona|
| 18|   2.1867|41.398258|       Av. Meridiana|
| 19| 2.18669

Об’єднайте (`JOIN`) вибрані критичні часові інтервали з таблицею станцій, щоб отримати координати станцій

In [44]:
# ваш код
# Реєструємо тимчасову таблицю для об'єднання
stations_df.createOrReplaceTempView("stations")

# Виконуємо SQL запит для об'єднання таблиці станцій з непорожніми результатами критичності
join_df = spark.sql("""
    SELECT s.*, t.*
    FROM stations s
    LEFT JOIN (
        SELECT *
        FROM selected_criticality_df
    ) t ON s.id = t.station
""")

# Виводимо результат
join_df.show()




+---+---------+---------+--------------------+-------+----------+-------+-------------------+
| id|longitude| latitude|                name|station| dayofweek|   hour|        criticality|
+---+---------+---------+--------------------+-------+----------+-------+-------------------+
|  1| 2.180019|41.397978|Gran Via Corts Ca...|      1|2023-09-29|13:24:0| 0.2962962962962963|
|  1| 2.180019|41.397978|Gran Via Corts Ca...|      1|2023-09-29| 6:18:0|0.37037037037037035|
|  1| 2.180019|41.397978|Gran Via Corts Ca...|      1|2023-09-29| 5:24:0| 0.4074074074074074|
|  1| 2.180019|41.397978|Gran Via Corts Ca...|      1|2023-09-26|12:28:0| 0.5454545454545454|
|  1| 2.180019|41.397978|Gran Via Corts Ca...|      1|2023-09-25| 4:44:0| 0.9047619047619048|
|  1| 2.180019|41.397978|Gran Via Corts Ca...|      1|2023-09-24|20:26:0| 0.5714285714285714|
|  1| 2.180019|41.397978|Gran Via Corts Ca...|      1|2023-09-24|17:12:0| 0.3333333333333333|
|  1| 2.180019|41.397978|Gran Via Corts Ca...|      1|2023-0

Відсортуйте вміст DataFrame

In [49]:
# ваш код
# Реєструємо тимчасову таблицю для обробки
join_df.createOrReplaceTempView("merged_table")

# Виконуємо SQL запит для вибору потрібних стовпців та впорядкування результату
sortedby_df = spark.sql("""
    SELECT station, dayofweek, hour, criticality, longitude, latitude
    FROM merged_table
""")

# Виводимо результат
sortedby_df.show()



+-------+----------+-------+-------------------+---------+---------+
|station| dayofweek|   hour|        criticality|longitude| latitude|
+-------+----------+-------+-------------------+---------+---------+
|      1|2023-09-29|13:24:0| 0.2962962962962963| 2.180019|41.397978|
|      1|2023-09-29| 6:18:0|0.37037037037037035| 2.180019|41.397978|
|      1|2023-09-29| 5:24:0| 0.4074074074074074| 2.180019|41.397978|
|      1|2023-09-26|12:28:0| 0.5454545454545454| 2.180019|41.397978|
|      1|2023-09-25| 4:44:0| 0.9047619047619048| 2.180019|41.397978|
|      1|2023-09-24|20:26:0| 0.5714285714285714| 2.180019|41.397978|
|      1|2023-09-24|17:12:0| 0.3333333333333333| 2.180019|41.397978|
|      1|2023-09-22| 22:6:0|               0.75| 2.180019|41.397978|
|      1|2023-09-22| 5:32:0|               0.85| 2.180019|41.397978|
|      1|2023-09-19|23:32:0| 0.9444444444444444| 2.180019|41.397978|
|      1|2023-09-19|15:16:0|  0.631578947368421| 2.180019|41.397978|
|      1|2023-09-19|  1:2:0| 0.952

Write to file:

In [50]:
# ваш код
# Реєструємо тимчасову таблицю для обробки
sortedby_df .createOrReplaceTempView("sorted_table")

# Виконуємо SQL запит для запису даних DataFrame у CSV-файл
spark.sql("""
    SELECT *
    FROM sorted_table
""").write.csv("ready_data.csv", header=True)


## Зупинка Spark

In [51]:
spark.stop()